In [23]:
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

# Importing CSV Files

In [5]:
df_BTC = pd.read_csv("BTC_Historical_11-03.csv", index_col=0)
df_DOGE = pd.read_csv("DOGE_Historical_11-03.csv", index_col=0)
df_XRP = pd.read_csv("XRP_Historical_11-03.csv", index_col=0)
df_ETH = pd.read_csv("ETH_Historical_11-03.csv", index_col=0)
df_UNI = pd.read_csv("UNI_Historical_11-03.csv", index_col=0)

# LINREQ

In [32]:
# BTC

In [30]:
degree = 2  # Degree of polynomial features
polyreg = make_pipeline(PolynomialFeatures(degree), LinearRegression())

# Features and target variable
X = df_BTC[['Closing Price (USD)']]
y = df_BTC['y']

# Fit the model
polyreg.fit(X, y)

# Make future predictions (let's say predicting for the next 30 days)
future_dates = pd.date_range(start='2024-03-12', periods=30, freq='D')
last_available_data = df_BTC.iloc[-1]
future_X = pd.DataFrame({'Closing Price (USD)': [last_available_data['Closing Price (USD)']]}, index=[future_dates[0]])
future_y = polyreg.predict(future_X)

# Print future predictions
print("Predicted highest prices for future dates:")
for date, price in zip(future_dates, future_y):
    print(f"{date}: {price}")

Predicted highest prices for future dates:
2024-03-12 00:00:00: 69853.49647791797


In [33]:
# DOGE

In [37]:
degree = 2

# Initialize polynomial regression model
polyreg = make_pipeline(PolynomialFeatures(degree), LinearRegression())

# Features and target variable
X = df_DOGE[['Closing Price (USD)']]
y = df_DOGE['Highest Price (USD)']

# Fit the model
polyreg.fit(X, y)

# Make future predictions (let's say predicting for the next 30 days)
future_dates = pd.date_range(start='2024-03-12', periods=30, freq='D')
future_X = pd.DataFrame({'Closing Price (USD)': [0.17]}, index=[future_dates[0]])
future_y = polyreg.predict(future_X)

# Print future predictions
print("Predicted highest prices for future dates:")
for date, price in zip(future_dates, future_y):
    print(f"{date}: {price}")

Predicted highest prices for future dates:
2024-03-12 00:00:00: 0.17887508057279983


In [38]:
# Etherum

In [40]:
degree = 2

# Initialize polynomial regression model
polyreg = make_pipeline(PolynomialFeatures(degree), LinearRegression())

# Features and target variable
X = df_ETH[['Closing Price (USD)']]
y = df_ETH['Highest Price (USD)']

# Fit the model
polyreg.fit(X, y)

# Make future predictions (let's say predicting for the next 30 days)
future_dates = pd.date_range(start='2024-03-12', periods=30, freq='D')
future_X = pd.DataFrame({'Closing Price (USD)': [3817.03]}, index=[future_dates[0]])
future_y = polyreg.predict(future_X)

# Print future predictions
print("Predicted highest prices for future dates:")
for date, price in zip(future_dates, future_y):
    print(f"{date}: {price}")

Predicted highest prices for future dates:
2024-03-12 00:00:00: 3915.4894684060555


In [41]:
# XRP

In [44]:
degree = 2

# Initialize polynomial regression model
polyreg = make_pipeline(PolynomialFeatures(degree), LinearRegression())

# Features and target variable
X = df_XRP[['Closing Price (USD)']]
y = df_XRP['Highest Price (USD)']

# Fit the model
polyreg.fit(X, y)

# Make future predictions (let's say predicting for the next 30 days)
future_dates = pd.date_range(start='2024-03-12', periods=30, freq='D')
future_X = pd.DataFrame({'Closing Price (USD)': [0.6]}, index=[future_dates[0]])
future_y = polyreg.predict(future_X)

# Print future predictions
print("Predicted highest prices for future dates:")
for date, price in zip(future_dates, future_y):
    print(f"{date}: {price}")

Predicted highest prices for future dates:
2024-03-12 00:00:00: 0.6186635649931228


In [46]:
# UNI

In [56]:
df_UNI

,Opening Price (USD),Highest Price (USD),Lowest Price (USD),Closing Price (USD),5. volume,6. market cap (USD),Difference between opening and closing
2021-06-16,23.42,23.55,21.64,21.83,1687477.96,1687477.96,-1.59
2021-06-17,21.83,22.94,21.40,22.18,1443998.15,1443998.15,0.35
2021-06-18,22.18,22.27,19.55,20.39,2187107.79,2187107.79,-1.79
2021-06-19,20.40,20.98,19.75,19.76,1744280.98,1744280.98,-0.64
2021-06-20,19.76,21.02,18.62,20.70,2410717.38,2410717.38,0.94
...,...,...,...,...,...,...,...
2024-03-07,15.41,15.57,14.53,14.88,6059680.43,6059680.43,-0.53
2024-03-08,14.88,15.45,14.16,14.58,5109719.80,5109719.80,-0.30
2024-03-09,14.58,15.01,13.84,14.18,5427317.52,5427317.52,-0.40
2024-03-10,14.19,14.42,13.50,14.23,4646319.95,4646319.95,0.04


In [60]:
df_afjafj = df_UNI["Highest Price (USD)"]

In [99]:
degree = 5


# Initialize polynomial regression model
polyreg = make_pipeline(PolynomialFeatures(degree), LinearRegression())

# Features and target variable
X = df_UNI[['Closing Price (USD)']]
y = df_UNI['Highest Price (USD)']

# Fit the model
polyreg.fit(X, y)

future_dates = pd.date_range(start='2024-03-12', periods=7, freq='D')
future_X = pd.DataFrame({'Closing Price (USD)': [14.13]}, index=[future_dates[0]])
future_y = polyreg.predict(future_X)

# Print future predictions
print("Predicted highest prices for future dates:")
for date, price in zip(future_dates, future_y):
    print(f"{date}: {price}")

Predicted highest prices for future dates:
2024-03-12 00:00:00: 14.94103422104355
